In [1]:
from main import *
from neural_network import *
import time
import json
from dataset import Dataset
from rise_and_fall import *
from IPython.display import clear_output
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils.vis_utils import plot_model
from matplotlib import pyplot as plt
import scipy as sci
np.set_printoptions(precision=3, linewidth=2*75, suppress=False)
import cProfile

In [2]:
db = Dataset()

In [3]:
# def get_minimalistic_frame(
#     at=1000,
#     length=576,
#     backwardlook=288,
#     max_size=64,
#     kinterval=5.0,
# ):
#     aa = []
#     size = 0
#     look_past = 0
#     mn_index = np.zeros((0,))
#     mx_index = np.zeros((0,))
#     end_at = at-look_past
#     max_size += 1
#     last_vectors = []

#     while size < max_size:
#         start_at = at-length-look_past
#         a = db.nf.iloc[start_at:end_at, :]
#         a.reset_index(drop=True, inplace=True)

#         mxis, mnis, ch = finfo(a, fee=0.005)
#         mn_index = np.append(mn_index, np.array(mnis)+start_at)
#         mx_index = np.append(mx_index, np.array(mxis)+start_at)

#         a = a.assign(mnis=0.0)
#         a = a.assign(mxis=0.0)
#         a = a.assign(ch=0.0)
#         a.loc[mnis, 'mnis'] = 1.0
#         a.loc[mxis, 'mxis'] = 1.0
#         a.loc[mxis, 'ch'] = np.array(ch)*100.0

#         fltr = np.logical_or(a['mnis'], a['mxis'])
#         indices = a[fltr].index
#         vectors = np.zeros((indices.shape[0], 7))
#         for i in range(indices.shape[0]-1):
#             vectors[i+1, [0, 2, 3]] = a.loc[0 if i == 0 else indices[i]
#                                                             :indices[i+1], ['meanp', 'taker', 'maker']].mean().to_numpy()
#             stdp = a.loc[0 if i == 0 else indices[i]
#                                             :indices[i+1], 'meanp'].std()
#             if np.isnan(stdp):
#                 stdp = 0.0
#             vectors[i+1, 1] = stdp
#             vectors[i+1, [4, 5]] = a.loc[0 if i == 0 else indices[i]
#                 :indices[i+1], ['vol', 'ntrds']].sum()
#             vectors[i+1, 6] = (indices[i+1]-indices[i])*kinterval

#         last_vector = np.zeros((7,))
#         last_vector[[0, 2, 3]] = a.loc[indices[-1]:,
#                                         ['meanp', 'taker', 'maker']].mean().to_numpy()
#         last_vector[1] = a.loc[indices[-1]:, 'meanp'].std()
#         last_vector[[4, 5]] = a.loc[indices[-1]:, ['vol', 'ntrds']].sum()
#         last_vector[6] = (a.index[-1]-indices[-1])*kinterval
#         last_vectors.append(last_vector)

#         a = a.loc[fltr, :]
#         a.reset_index(drop=True, inplace=True)
#         a = pd.concat([a.loc[:, ['stime', 'meanp', 'stdp', 'ch']], pd.DataFrame(
#             vectors, columns=['avgp', 'stdavgp', 'taker', 'maker', 'vol', 'ntrds', 'gap'])], axis=1)
#         aa.append(a)
#         size += a.shape[0]
#         look_past += backwardlook
#         end_at = start_at

#     for i in range(len(last_vectors)-1):
#         aa[i].loc[0, ['avgp', 'stdavgp', 'taker', 'maker',
#                         'vol', 'ntrds', 'gap']] = last_vectors[i+1]
#     aa.reverse()
#     a = pd.concat(aa)
#     index = a.pop('stime')
#     a.set_index(pd.to_datetime(index, unit='ms'), drop=True, inplace=True)

#     return a.iloc[-(max_size-1):], mx_index, mn_index

# def get_labled_frame_for_buy(
#     db,
#     at=80000,
#     length=576,
#     backwardlook=288,
#     max_size=64,
#     kinterval=5.0,
# ):
#     a, mxis, _ = db.get_minimalistic_frame(
#         at=at, length=length, backwardlook=backwardlook, max_size=max_size-1, kinterval=kinterval)
#     mxi = int(mxis.max())
#     mnis = []
#     inc = 0
#     while len(mnis) < 2:
#         next_mxis, mnis, next_chs = finfo(
#             db.nf.loc[mxi:mxi+48+inc], fee=0.002)
#         inc += 12
#     next_mni = mxi+mnis[0]
#     next_mxi = mxi+next_mxis[0]
#     next_ch = next_chs[0]
#     if next_mni-mxi < 2:
#         next_mni = mxi+mnis[1]
#         next_mxi = mxi+next_mxis[0]
#         next_ch = next_chs[1]
#     random_mni = np.random.randint(mxi+1, next_mni)
#     mni = (next_mni-random_mni)*kinterval
#     next_mxi = (next_mxi-random_mni)*kinterval

#     vector = np.zeros((a.shape[1],))
#     ft = db.nf.iloc[mxi:random_mni, :]
#     vector[[0, 1]] = ft.loc[ft.index[-1], ['meanp', 'stdp']]
#     vector[[3, 5, 6]] = ft.loc[:, ['meanp', 'taker', 'maker']].mean()
#     meanpstd = ft.loc[:, 'meanp'].std()
#     if np.isnan(meanpstd):
#         meanpstd = 0.0
#     vector[4] = meanpstd
#     vector[[7, 8]] = ft.loc[:, ['vol', 'ntrds']].sum()
#     vector[9] = ft.shape[0]*kinterval
#     ctime = pd.to_datetime(ft['stime'].iloc[-1], unit='ms')

#     a = np.concatenate([a, vector.reshape(1, -1)], axis=0)
#     ctime = ctime.weekday()*288 + (ctime.hour*60.0 + ctime.minute)/5.0
#     return a, ctime, mni, next_mxi, next_ch*100.0

In [44]:
def get_minimalistic_frame(
    at=1000,
    length=144,
    backwardlook=144,
    max_size=64,
    kinterval=5.0,
):
    aa = []
    size = 0
    look_past = 0
    mn_index = np.zeros((0,))
    mx_index = np.zeros((0,))
    end_at = at-look_past
    max_size += 1
    last_vectors = []
    first_vectors = []

    while size < max_size:
        start_at = at-length-look_past
        a = db.nf.iloc[start_at:end_at, :]
        a.reset_index(drop=True, inplace=True)

        mxis, mnis, ch = finfo(a, fee=0.005)
        mn_index = np.append(mn_index, np.array(mnis)+start_at)
        mx_index = np.append(mx_index, np.array(mxis)+start_at)

        a = a.assign(mnis=0.0)
        a = a.assign(mxis=0.0)
        a = a.assign(ch=0.0)
        a.loc[mnis, 'mnis'] = 1.0
        a.loc[mxis, 'mxis'] = 1.0
        a.loc[mxis, 'ch'] = np.array(ch)*100.0

        fltr = np.logical_or(a['mnis'], a['mxis'])
        indices = a[fltr].index
        vectors = np.zeros((indices.shape[0], 7))

        first_vector = np.zeros((7,))
        first_vector[[0, 2, 3]] = a.loc[:indices[0],['meanp', 'taker', 'maker']].mean().to_numpy()
        first_vector[1] = a.loc[:indices[0], 'meanp'].std()
        first_vector[[4, 5]] = a.loc[:indices[0], ['vol', 'ntrds']].sum()
        first_vector[6] = (indices[0]-a.index[0])*kinterval
        first_vectors.append(first_vector)

        for i in range(indices.shape[0]-1):
            vectors[i+1, [0, 2, 3]] = a.loc[indices[i]:indices[i+1], ['meanp', 'taker', 'maker']].mean().to_numpy()
            stdp = a.loc[indices[i]:indices[i+1], 'meanp'].std()
            if np.isnan(stdp):
                stdp = 0.0
            vectors[i+1, 1] = stdp
            vectors[i+1, [4, 5]] = a.loc[indices[i]:indices[i+1], ['vol', 'ntrds']].sum()
            vectors[i+1, 6] = (indices[i+1]-indices[i])*kinterval


        last_vector = np.zeros((7,))
        last_vector[[0, 2, 3]] = a.loc[indices[-1]:,['meanp', 'taker', 'maker']].mean().to_numpy()
        last_vector[1] = a.loc[indices[-1]:, 'meanp'].std()
        last_vector[[4, 5]] = a.loc[indices[-1]:, ['vol', 'ntrds']].sum()
        last_vector[6] = (a.index[-1]-indices[-1])*kinterval
        last_vectors.append(last_vector)

        a = a.loc[fltr, :]
        a.reset_index(drop=True, inplace=True)
        a = pd.concat([a.loc[:, ['stime', 'meanp', 'stdp', 'ch']], pd.DataFrame(
            vectors, columns=['avgp', 'stdavgp', 'taker', 'maker', 'vol', 'ntrds', 'gap'])], axis=1)
        aa.append(a)
        size += a.shape[0]
        look_past += backwardlook
        end_at = start_at

    for i in range(len(last_vectors)-1):
        aa[i].loc[0, ['avgp', 'stdavgp', 'taker', 'maker',
                      'vol', 'ntrds', 'gap']] = np.array([first_vectors[i],last_vectors[i+1]]).mean()
    aa.reverse()
    a = pd.concat(aa)
    index = a.pop('stime')
    a.set_index(pd.to_datetime(index, unit='ms'), drop=True, inplace=True)
    return a.iloc[-(max_size-1):], mx_index, mn_index


def get_labled_frame_for_buy(
    at=80000,
    length=576,
    backwardlook=288,
    max_size=64,
    kinterval=5.0,
):
    a, mxis, _ = db.get_minimalistic_frame(
        at=at, length=length, backwardlook=backwardlook, max_size=max_size-1, kinterval=kinterval)
    mxi = int(mxis.max())
    mnis = []
    inc = 0
    while len(mnis) < 2:
        next_mxis, mnis, next_chs = finfo(
            db.nf.loc[mxi:mxi+48+inc], fee=0.002)
        inc += 12
    next_mni = mxi+mnis[0]
    next_mxi = mxi+next_mxis[0]
    next_ch = next_chs[0]
    if next_mni-mxi < 2:
        next_mni = mxi+mnis[1]
        next_mxi = mxi+next_mxis[0]
        next_ch = next_chs[1]
    random_mni = np.random.randint(mxi+1, next_mni)
    mni = (next_mni-random_mni)*kinterval
    next_mxi = (next_mxi-random_mni)*kinterval

    vector = np.zeros((a.shape[1],))
    ft = db.nf.iloc[mxi:random_mni, :]
    vector[[0, 1]] = ft.loc[ft.index[-1], ['meanp', 'stdp']]
    vector[[3, 5, 6]] = ft.loc[:, ['meanp', 'taker', 'maker']].mean()
    meanpstd = ft.loc[:, 'meanp'].std()
    if np.isnan(meanpstd):
        meanpstd = 0.0
    vector[4] = meanpstd
    vector[[7, 8]] = ft.loc[:, ['vol', 'ntrds']].sum()
    vector[9] = ft.shape[0]*kinterval
    ctime = pd.to_datetime(ft['stime'].iloc[-1], unit='ms')

    a = np.concatenate([a, vector.reshape(1, -1)], axis=0)
    ctime = ctime.weekday()*288 + (ctime.hour*60.0 + ctime.minute)/5.0
    return a, ctime, mni, next_mxi, next_ch*100.0


In [32]:
f,index = get_minimalistic_frame()
col = 'meanp'
np.array([f[col][index[i-1]:index[i]].mean() for i in range(1,index.shape[0])])

(15, 3)
(17, 3)
(21, 3)
(19, 3)


ValueError: too many values to unpack (expected 2)

In [46]:
a,_,_=get_minimalistic_frame()
a

,meanp,stdp,ch,avgp,stdavgp,taker,maker,vol,ntrds,gap
stime,,,,,,,,,,
2017-08-18 21:40:00,3961.7675,17.924905,0.000000,3980.244000,14.331373,0.702364,0.297636,13773.740411,43.0,20.0
2017-08-18 23:20:00,4133.9025,2.428379,4.344904,4056.233214,56.315299,0.850572,0.101809,209338.047555,359.0,100.0
2017-08-19 00:20:00,4105.8825,22.276152,0.000000,4114.127885,7.729019,0.851668,0.148332,221554.929654,267.0,60.0
2017-08-19 00:25:00,4129.5975,5.000722,0.577586,4117.740000,16.769037,0.602075,0.397925,26858.019453,31.0,5.0
2017-08-19 00:35:00,4118.8075,2.385000,0.000000,4123.367500,5.585490,0.771768,0.228232,39940.739294,35.0,10.0
...,...,...,...,...,...,...,...,...,...,...
2017-08-20 11:20:00,4125.4400,0.000000,0.812893,4108.807500,23.521907,0.800422,0.199578,574.648856,6.0,5.0
2017-08-20 11:30:00,4094.0950,16.270432,0.000000,4114.991667,18.097044,0.760937,0.239063,19687.287388,21.0,10.0
2017-08-20 12:10:00,4125.4300,0.000000,0.765371,4103.075000,9.657196,0.641112,0.247777,69397.611350,103.0,40.0


In [47]:
def wrapper():
    for _ in range(50):
        # a = db.nf.iloc[80000:80500, :]
        # a.reset_index(drop=True, inplace=True)
        # mxis, mnis, ch = finfo(a, fee=0.005)

        get_minimalistic_frame()

        # get_labled_frame_for_buy()
wrapper()
#1.2s
#13.3s #culprit
#13.1s

In [ ]:
def plot_frame(f: pd.DataFrame, mxis=[], mnis=[], mark=None):
    ps = f["meanp"].to_numpy()
    mn = ps.min()
    mx = ps.max()
    plt.plot(ps)
    if len(mxis) > 0:
        plt.vlines(x=mxis, ymin=mn, ymax=mx, colors='green',)
    elif 'mxis' in f.columns:
        plt.vlines(x=f.index[f['mxis'] > 0], ymin=mn, ymax=mx, colors='green',)
    if len(mnis) > 0:
        plt.vlines(x=mnis, ymin=mn, ymax=mx, colors='red',)
    elif 'mnis' in f.columns:
        plt.vlines(x=f.index[f['mnis'] > 0], ymin=mn, ymax=mx, colors='red',)
    if mark is not None:
        plt.vlines(x=[mark], ymin=mn, ymax=mx, colors='blue',)
    plt.show()


def unique(xs):
    ys = []
    for x in xs:
        if x in ys:
            continue
        ys.append(x)
    return ys

In [ ]:
a, mxis, mnis = get_minimalistic_frame()
a['meanp'].plot()